In [5]:
import pandas as pd
import numpy as np
from io import StringIO

csv_data = """player,games_played,pass_attempts,pass_yards,pass_tds,pass_ints,targets,rec_yards,rec_tds,sacks,tackles,def_ints,safeties,fg_attempts,fg_made,long_fg
Alex,8,60,900,12,4,30,420,6,2,10,1,0,0,0,0
Blake,6,0,0,0,0,40,520,8,1,8,0,0,0,0,0
Casey,4,25,260,3,2,18,210,2,0,5,2,1,6,5,42
Drew,9,0,0,0,0,28,310,4,3,14,1,0,0,0,0
Evan,7,80,980,10,6,22,260,3,1,6,0,0,0,0,0
Flynn,5,0,0,0,0,35,410,5,0,7,0,0,8,7,38
Gabe,10,0,0,0,0,26,290,3,4,18,2,0,0,0,0
Hayden,3,0,0,0,0,12,120,1,0,3,0,0,0,0,0
"""

df = pd.read_csv(StringIO(csv_data))

mvp_map = {
    "Alex": 3,
    "Blake": 2,
    "Casey": 1,
    "Drew": 2,
    "Evan": 3,
    "Flynn": 1,
    "Gabe": 2,
    "Hayden": 0,
}
df["mvp_awards"] = df["player"].map(mvp_map).fillna(0).astype(int)

df


,player,games_played,pass_attempts,pass_yards,pass_tds,pass_ints,targets,rec_yards,rec_tds,sacks,tackles,def_ints,safeties,fg_attempts,fg_made,long_fg,mvp_awards
0,Alex,8,60,900,12,4,30,420,6,2,10,1,0,0,0,0,3
1,Blake,6,0,0,0,0,40,520,8,1,8,0,0,0,0,0,2
2,Casey,4,25,260,3,2,18,210,2,0,5,2,1,6,5,42,1
3,Drew,9,0,0,0,0,28,310,4,3,14,1,0,0,0,0,2
4,Evan,7,80,980,10,6,22,260,3,1,6,0,0,0,0,0,3
5,Flynn,5,0,0,0,0,35,410,5,0,7,0,0,8,7,38,1
6,Gabe,10,0,0,0,0,26,290,3,4,18,2,0,0,0,0,2
7,Hayden,3,0,0,0,0,12,120,1,0,3,0,0,0,0,0,0


In [6]:
def participation_factor(games_played, cap=6):
    return min(1.0, games_played / cap) if games_played > 0 else 0.0

def passing_score(row):
    att = row["pass_attempts"]
    if att <= 0:
        return 0.0
    return (row["pass_yards"]/att)*0.5 + (row["pass_tds"]/att)*20 - (row["pass_ints"]/att)*15

def receiving_score(row):
    tgt = row["targets"]
    if tgt <= 0:
        return 0.0
    return (row["rec_yards"]/tgt)*0.5 + (row["rec_tds"]/tgt)*20

def offense_contribution(row):
    # QB eligibility rule
    if row["pass_attempts"] >= 20:
        return (passing_score(row)*1.3 + receiving_score(row)) / 2
    return receiving_score(row)

def defense_contribution(row):
    gp = row["games_played"]
    if gp <= 0:
        return 0.0
    return (
        (row["sacks"]/gp)*3 +
        (row["tackles"]/gp)*0.5 +
        (row["def_ints"]/gp)*6 +
        (row["safeties"]/gp)*10
    )

def special_teams_contribution(row):
    gp = row["games_played"]
    att = row["fg_attempts"]
    if gp <= 0 or att < 5:
        return 0.0
    accuracy = (row["fg_made"]/att)*10
    range_bonus = max(0, row["long_fg"] - 30)*0.05
    return accuracy + (row["fg_made"]/gp) + range_bonus

# Compute components
df["pf"] = df["games_played"].apply(participation_factor)
df["offense"] = df.apply(offense_contribution, axis=1)
df["defense"] = df.apply(defense_contribution, axis=1)
df["special"] = df.apply(special_teams_contribution, axis=1)

df[["player","offense","defense","special","pf","mvp_awards"]]


,player,offense,defense,special,pf,mvp_awards
0,Alex,12.325000,2.125000,0.000000,1.000000,3
1,Blake,10.500000,1.166667,0.000000,1.000000,2
2,Casey,8.187778,6.125000,10.183333,0.666667,1
3,Drew,8.392857,2.444444,0.000000,1.000000,2
4,Evan,9.193182,0.857143,0.000000,1.000000,3
5,Flynn,8.714286,0.700000,10.550000,0.833333,1
6,Gabe,7.884615,3.300000,0.000000,1.000000,2
7,Hayden,6.666667,0.500000,0.000000,0.500000,0


In [7]:
def zscore(series):
    if series.std() == 0:
        return series * 0
    return (series - series.mean()) / series.std()

# Target: MVP rate (MVPs per game)
df["target"] = df["mvp_awards"] / df["games_played"]

Xo = zscore(df["offense"])
Xd = zscore(df["defense"])
Xs = zscore(df["special"])
pf = df["pf"]

best_score = -1
best_weights = None
grid = [0.5, 1.0, 1.5, 2.0]

for wo in grid:
    for wd in grid:
        for ws in grid:
            score = pf * (wo*Xo + wd*Xd + ws*Xs)
            ranked = df.assign(score=score).sort_values("score", ascending=False)

            # Evaluation metric: average MVP rate of top 3 ranked players
            eval_metric = ranked.head(3)["target"].mean()

            if eval_metric > best_score:
                best_score = eval_metric
                best_weights = (wo, wd, ws)

print("Best weights (offense, defense, special):", best_weights)
print("Best top-3 MVP-rate score:", best_score)

# Apply best weights
wo, wd, ws = best_weights
df["final_score"] = pf * (wo*Xo + wd*Xd + ws*Xs)

final_ranked = df.sort_values("final_score", ascending=False)[
    ["player","games_played","mvp_awards","target","final_score","offense","defense","special"]
]

final_ranked


Best weights (offense, defense, special): (1.0, 0.5, 0.5)
Best top-3 MVP-rate score: 0.3194444444444444


,player,games_played,mvp_awards,target,final_score,offense,defense,special
0,Alex,8,3,0.375000,1.645876,12.325000,2.125000,0.000000
2,Casey,4,1,0.250000,0.927465,8.187778,6.125000,10.183333
1,Blake,6,2,0.333333,0.340464,10.500000,1.166667,0.000000
5,Flynn,5,1,0.200000,0.239662,8.714286,0.700000,10.550000
4,Evan,7,3,0.428571,-0.493973,9.193182,0.857143,0.000000
3,Drew,9,2,0.222222,-0.531790,8.392857,2.444444,0.000000
6,Gabe,10,2,0.200000,-0.596406,7.884615,3.300000,0.000000
7,Hayden,3,0,0.000000,-1.021481,6.666667,0.500000,0.000000


In [8]:
final_ranked.to_csv("rankings.csv", index=False)
print("Saved rankings.csv")


Saved rankings.csv
